In [ ]:
# !pip install nltk

In [ ]:
# !pip install zeyrek

In [ ]:
# !pip install TurkishStemmer

In [ ]:
import nltk
import zeyrek # https://github.com/obulat/zeyrek
from TurkishStemmer import TurkishStemmer # https://github.com/otuncelli/turkish-stemmer-python

In [ ]:
nltk.download("punkt")

In [ ]:
analyzer = zeyrek.MorphAnalyzer()

In [ ]:
analyzer.lemmatize("benimkilerden")

In [ ]:
stemmer = TurkishStemmer()

In [ ]:
stemmer.stem("benimkilerden")

In [ ]:
cumle = "Dönülmez akşamın ufkundayım."

In [ ]:
stemmer.stem(cumle)

In [ ]:
analyzer.lemmatize(cumle)

In [ ]:
for kelime in cumle.split():
    print(stemmer.stem(kelime))

In [ ]:
import pandas as pd

In [ ]:
# !pip install openpyxl

In [ ]:
df = pd.read_excel("filmveriseti.xlsx") 
# https://www.kaggle.com/code/furkanyeter/content-based-recommendation-system-turkish-movie/data
df.head()

In [ ]:
len(df)

In [ ]:
df["tur"].unique()

In [ ]:
df.info()

In [ ]:
df.isna().sum()

In [ ]:
nltk.download("stopwords")

In [ ]:
import string
from nltk.corpus import stopwords

In [ ]:
string.punctuation

In [ ]:
dolgular = stopwords.words("turkish")
dolgular

In [ ]:
s = "Hey! Naber? İyilik, senden... Ben de iyiyim."
s = s.translate(str.maketrans ('', '', string.punctuation))

In [ ]:
s

In [ ]:
analiz = analyzer.lemmatize(s)
analiz

In [ ]:
for el in analiz:
    print(el[1][-1])

In [ ]:
ornek = df.iloc[13]["ozet ve detaylar"]
ornek

In [ ]:
ornek.translate(str.maketrans ('', '', string.punctuation))

In [ ]:
ornek = "film,çok güzel.tavsiye ederim."
ornek.translate(str.maketrans ('', '', string.punctuation))

In [ ]:
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r"\w+")
tokenizer.tokenize(ornek)

In [ ]:
def clean_doc(doc, tok):
    """
    Args:
        doc (str): Köklerine ayrılacak doküman.
        tok (RegexpTokenizer): Instance of RegexpTokenizer from nltk.tokenize.

    Returns:
        [str]: List of lemmatized words from the doc argument without punctuation and stop words.
    
    Ex: clean_doc("film,çok güzel.tavsiye ederim.", tokenizer) -> ['film', 'güzel', 'tavsiye', 'eder']
    """
    tokens = tok.tokenize(doc) # doc'ları kelimelere ayır, noktalama işaretlerini çıkar
    clean_tokens = [tok for tok in tokens if tok not in dolgular] # stop word'leri ayıkla
    analiz = analyzer.lemmatize(" ".join(clean_tokens)) # token'ları ayıkla
    return [el[1][-1] for el in analiz] # temizlenmiş token'ları liste olarak döndür

In [ ]:
tokenizer = RegexpTokenizer(r"\w+")
clean_doc(ornek, tokenizer)

In [ ]:
df.head()

In [ ]:
# bu işlem uzun sürebilir
# uzayda bir kavanozun kapağını açıp kapattığınızda ve kavanozu Ay'a doğru
# fırlattığınızda kavanoz içindeki boşluk hareket eder mi?
import ast

def extract_features(row, tokenizer):
    ozet = clean_doc(row["ozet ve detaylar"], tokenizer)
    tur = row["tur"]
    return ozet + ast.literal_eval(tur)

df["clean"] = df.apply(lambda row: extract_features(row, tokenizer), axis=1) 

In [ ]:
df.head()

In [ ]:
import random
for i in range(3):
    rand_idx = random.randrange(len(df))
    ozet = df.iloc[rand_idx]["ozet ve detaylar"]
    print("Orijinal:")
    print(ozet + "\n")
    print("Temizlenmiş:")
    print(" ".join(df.iloc[rand_idx]["clean"]) + "\n")  
    print("/" * 50 + "\n")  

In [ ]:
df.to_csv("filmler_clean.csv", index=False)

In [ ]:
df_diskten = pd.read_csv("filmler_clean.csv")

In [ ]:
df_diskten.head()

In [ ]:
kelime2index = {}
index2kelime = {}

In [ ]:
kelime2index["<UNK>"] = 0
index2kelime[0] = "<UNK>"

In [ ]:
clean_col = df["clean"]
clean_col

In [ ]:
len(clean_col.values), len(df)

In [ ]:
i = 1
for doc in clean_col.values:
    for token in doc:
        if token in kelime2index:
            continue
        
        kelime2index[token] = i
        index2kelime[i] = token
        i += 1

In [ ]:
kelime2index

In [ ]:
index2kelime

In [ ]:
len(kelime2index)

In [ ]:
import numpy as np

In [ ]:
doc_mat = np.zeros((len(df), len(kelime2index)))

In [ ]:
for i, doc in enumerate(clean_col.values):
    for token in doc:
        token_index = kelime2index[token]
        doc_mat[i][token_index] += 1

In [ ]:
doc_mat[0]

In [ ]:
def cos_sim(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

In [ ]:
# sorgular
random.seed(42) # otostopçunun galaksi rehberi
film_indexi = random.randrange(len(df))
film = df.iloc[film_indexi]
film["film adi"], film["ozet ve detaylar"], film["tur"]

In [ ]:
benzerlikler = []
sorgu_film = doc_mat[film_indexi]
for vec in doc_mat:
    benzerlik = cos_sim(sorgu_film, vec)
    benzerlikler.append(benzerlik)

In [ ]:
test_list = [1, 5, 4, 2]
print(np.array(test_list).argsort()[:2])
print((- np.array(test_list)).argsort()[:2])

In [ ]:
# en yakın 5 film
top5_film_indexleri = (- np.array(benzerlikler)).argsort()[:5]

In [ ]:
for i in top5_film_indexleri:
    film = df.iloc[i]
    print(film["film adi"], film["ozet ve detaylar"], film["tur"])
    print(" - " * 20)

In [ ]:
def print_film(row_index):
    film = df.iloc[row_index]
    adi = film['film adi']
    turu = ", ".join(ast.literal_eval(film['tur']))
    ozet = film['ozet ve detaylar']
    if len(ozet) > 150:
        ozet = ozet[:150] + "..."

    print(f"Film adı: {adi} \nTürü: {turu} \nÖzeti: {ozet}")

In [ ]:
def benzerlerini_bul(film_adi, benzer_film_sayisi):
    film_indexi = df[df["film adi"] == film_adi].index[0]
    print("Aranan Film:")
    print_film(film_indexi)
    print(" * " * 20)
    print(" * " * 20)

    benzerlikler = []
    sorgu_film = doc_mat[film_indexi]
    for vec in doc_mat:
        benzerlik = cos_sim(sorgu_film, vec)
        benzerlikler.append(benzerlik)
    
    en_benzer_film_indexleri = (- np.array(benzerlikler)).argsort()[:benzer_film_sayisi + 1]
    print("Benzer Filmler:\n")
    for i in en_benzer_film_indexleri[1:]:
        print_film(i)
        print(" - " * 20)
        print(" - " * 20)

In [ ]:
benzerlerini_bul("Kirpi Sonic 2", 3)

İyileştirme için neler yapılabilir? (diğer sütunlar? sütun ağırlıkları?)

In [ ]:
df.head()